<a href="https://colab.research.google.com/github/nitchie/notebook_NK225op/blob/master/225op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:

import os
import re
import requests
from bs4 import BeautifulSoup
from google.colab import drive

def debug_print(str):
  print(str)
  #pass

#https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory
drive.mount('/content/drive')

storage_dir =  'drive/My Drive/225op/'

page_urls = [
    'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/index.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-01.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-02.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-03.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-04.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-05.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-06.html'
    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-07.html'
#    ,'https://www.jpx.co.jp/markets/derivatives/perticipant-volume/archives-11.html'
]
csvfile_list = []

for page_url in page_urls:
  soup = BeautifulSoup(requests.get(page_url).content,'html.parser')
  elements = soup.find_all(href=re.compile("csv"))
  if not elements:
    debug_print('page '+page_url+' is not found.')
  for el in elements:
    csv_url = el['href']
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_night.csv
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_night_J-NET.csv
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_whole_day.csv
    #/markets/derivatives/perticipant-volume/nlsgeu0000034wa2-att/20180531_volume_by_participant_whole_day_J-NET.csv
    m = re.match(r'/markets/derivatives/participant-volume/.+/(.+)',csv_url)
    if m:
      csvfile = m[1]
      csvfile_list = csvfile_list + [csvfile]
      outfile = storage_dir+csvfile
      
      if not os.path.exists(outfile):
          with open(outfile,'w') as f:
            if f:
                response = requests.get('https://www.jpx.co.jp'+csv_url)
                f.write(response.text)
                debug_print('save '+outfile)
    #break
    


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
save drive/My Drive/225op/20181031_volume_by_participant_night.csv
save drive/My Drive/225op/20181031_volume_by_participant_night_J-NET.csv
save drive/My Drive/225op/20181031_volume_by_participant_whole_day.csv
save drive/My Drive/225op/20181031_volume_by_participant_whole_day_J-NET.csv
save drive/My Drive/225op/20181030_volume_by_participant_night.csv
save drive/My Drive/225op/20181030_volume_by_participant_night_J-NET.csv
save drive/My Drive/225op/20181030_volume_by_participant_whole_day.csv
save drive/My Drive/225op/20181030_volume_by_participant_whole_day_J-NET.csv
save drive/My Drive/225op/20181029_volume_by_participant_night.csv
save drive/My Drive/225op/20181029_volume_by_participant_night_J-NET.csv
save drive/My Drive/225op/20181029_volume_by_participant_whole_day.csv
save drive/My Drive/225op/20181029_volume_by_participant_whole_day_J-NET.csv
save dr

In [0]:
  import csv 
  from pandas import DataFrame,Series
  
  jpxcodes = {}
  companys = {}
  positions = {}
  cnt=0
  
  def add_record(company_code,company_name_j,company_name_e,volume,instrument,tradedate,ls,daynight,market,cnt):
    companys[company_code] = {'company_name_j':company_name_j,'company_name_e':company_name_e}

    s = instrument.split('_')
    kubun = s[0]
    product = s[1]
    period = s[2]
    if kubun != 'FUT':
      price = s[3]
    else:
      price = ''
    if ls == 'short':
      volume = volume * -1
    #if tradedate> "20190405":
    #  return
    positions[cnt] = {'tradedate':tradedate,'daynight':daynight,'market':market,'company_name_e':company_name_e,'kubun':kubun,'product':product,'period':period,'price':price,'ls':ls,'volume':volume}
                  
  for csvfile in csvfile_list:
    with open(storage_dir+csvfile) as f:
      if f:
        lines = list(csv.reader(f))
        jpxcode = ''
        instrument = ''
        tradedate = lines[1][3]
        daynight = 'day'
        if re.search(r'night',csvfile):
          daynight = 'night'
        market='market'
        if re.search(r'J\-NET',csvfile):
          market='J-NET'
        if re.search(r'\d{8}',tradedate):
          # debug_print(csvfile+" "+tradedate)
          for line in lines:
            if line:
              if line[0] == 'JPX Code':
                jpxcode = line[1]
              elif line[0] == 'Instrument':
                instrument = line[1]
                jpxcodes[jpxcode] = instrument
              else:
                if re.search(r'^\d+$',line[0]):
                  cnt = cnt + 1
                  add_record(line[0],line[1],line[2],int(line[3]),instrument,tradedate,'short',daynight,market,cnt)
                if len(line) > 4 and re.search(r'^\d+$',line[4]):
                  cnt = cnt + 1
                  add_record(line[4],line[5],line[6],int(line[7]),instrument,tradedate,'long',daynight,market,cnt)
        else:
          pass # show error


In [0]:
from pandas import DataFrame,Series
df = DataFrame(positions)
#df.loc['ABN AMRO Clearing','FUT_NK225_1906']
df2 = df.T.set_index(['company_name_e','kubun','product','period','price','ls'])


In [74]:
#df2.query('product == "NK225" & period =="190412"　').sum(level=['company_name_e','kubun','product','period','price']).sort_values(by=['company_name_e','product','period','kubun','price'], ascending=[True,True,False,True,False]).drop(columns=['daynight','market','tradedate'])
#df2.query('product == "NK225" & period == "190308" & kubun == "PUT" & price == "21500" & tradedate <= "20190301"').sum(level=['company_name_e','kubun','product','period','price']).sort_values(by=['company_name_e','product','period','kubun','price'], ascending=[True,True,False,True,False]).drop(columns=['daynight','market','tradedate'])
df2.query('product == "NK225" ').groupby("period").size()
#df2


period
1809       1198
180914    11981
181012    23845
181109    25324
1812       9261
181214    28195
190111    18145
190208    21780
1903      11153
190308    24182
190412    29680
190510     7350
1906       7111
1909       1296
dtype: int64

[JPX Code] [instrument name]

[company code] [company name]
[tradedate+day/night+market/j-net+jpxcode+companycode+long/short] [volume]

instrument,companyによる絞り込み
    companyは単純一致でOKだが、instrumentは部分一致で絞り込みになる

インデックスっぽいカラムを沢山作ってしまって、あとでデータをグルーピングしたりできるものだろうか
    day/nightやmarket/j-netなどは基本合算しかしなさそう
    long/shortなども合算してnetの数値が見たいのが基本

In [45]:
companys

{'11004': {'company_name_e': 'ARK SECURITIES', 'company_name_j': 'アーク証券'},
 '11016': {'company_name_e': 'AIZAWA SECURITIES', 'company_name_j': '藍澤證券'},
 '11056': {'company_name_e': 'ANDO SECURITIES', 'company_name_j': '安藤証券'},
 '11060': {'company_name_e': 'kabu.com Securities',
  'company_name_j': 'カブドットコム証券'},
 '11168': {'company_name_e': 'Imamura Securities', 'company_name_j': '今村証券'},
 '11212': {'company_name_e': 'ACE SECURITIES', 'company_name_j': 'エース証券'},
 '11216': {'company_name_e': 'Eiwa Securities', 'company_name_j': '永和証券'},
 '11256': {'company_name_e': 'SBI SECURITIES', 'company_name_j': 'ＳＢＩ証券'},
 '11272': {'company_name_e': 'OKASAN SECURITIES', 'company_name_j': '岡三証券'},
 '11280': {'company_name_e': 'OKACHI SECURITIES', 'company_name_j': '岡地証券'},
 '11448': {'company_name_e': 'H.S. SECURITIES', 'company_name_j': 'エイチ・エス証券'},
 '11456': {'company_name_e': 'Kyowa Securities', 'company_name_j': '共和証券'},
 '11464': {'company_name_e': 'KYOKUTO SECURITIES', 'company_name_j': '極東証券'

In [25]:
import pandas as pd
print(pd.__version__)
pd.show_versions()

0.23.4


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)



INSTALLED VERSIONS
------------------
commit: None
python: 3.6.7.final.0
python-bits: 64
OS: Linux
OS-release: 4.14.79+
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.23.4
pytest: 3.6.4
pip: 19.0.3
setuptools: 40.9.0
Cython: 0.29.6
numpy: 1.16.2
scipy: 1.2.1
pyarrow: None
xarray: 0.11.3
IPython: 5.5.0
sphinx: 1.8.5
patsy: 0.5.1
dateutil: 2.5.3
pytz: 2018.9
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.9
feather: None
matplotlib: 3.0.3
openpyxl: 2.5.9
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: None
lxml: 4.2.6
bs4: 4.6.3
html5lib: 1.0.1
sqlalchemy: 1.3.2
pymysql: None
psycopg2: 2.7.6.1 (dt dec pq3 ext lo64)
jinja2: 2.10.1
s3fs: 0.2.0
fastparquet: None
pandas_gbq: 0.4.1
pandas_datareader: 0.7.0
